<img src="http://vision.skills.network/logo-light.png" width="400" alt="CV Studio logo"  />


<img src="https://keras.io/img/logo.png">


Estimated time needed: **60** minutes


<h2> Image Classification with Convolution Neural Network (CNN) using  Keras and Tensorflow</h2>
<h3>Project: Final_project_stop_signs</h3>
<h3>Training Run: Train</h3>


A Convolutional Neural Network is a type of neural network that is used to train models using images.


<ul>
    <li>Table of Content
        <ul>
            <li>Download Images and Annotations</li>
            <li>Build the Model</li>
            <li>Train the Model</li>
            <li>Model Performance</li>
            <li>Report Results</li>
        </ul>
    </li>
    
</ul>


----


## Load Important Libraries


In [ ]:
#if you get an error in your environment please uncomment to install neccessary packages
# !pip install imutils
# !pip install skillsnetwork
# !pip install tensorflow==1.15

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import set_random_seed
import os
from os import path
from datetime import datetime
import shutil
import random
from skillsnetwork import cvstudio

## Download Your Images and Annotations


Now let's download the images from CV Studio.

If you have run this section before, you don't need to run it again. Feel free to jump to the <b>Build the Model</b> section.


In [ ]:
# Initialize the CV Studio Client
cvstudioClient = cvstudio.CVStudio()

# Download All Images
cvstudioClient.downloadAll()

In [ ]:
# Get the annotations from CV Studio
annotations = cvstudioClient.get_annotations()
num_classes = len(annotations['labels'])

In [ ]:
# Move files into folders

def labeldir(label):
    return path.join(os.getcwd(), 'images', label)

for label in annotations['labels']:
    if not path.exists(labeldir(label)):
        os.mkdir(labeldir(label))

for key, value in annotations['annotations'].items():
    src = path.join(os.getcwd(), 'images', key)
    if path.exists(src):
        dest = (path.join(labeldir(value[0]['label']), key))
        shutil.move(src, dest)

## Build the Model

We start with a MobileNetV2 architecture as the backbone pretrained feature extractor. We then add a couple of dense layers and a softmax layer to perfom the classification. We freeze the MobileNetV2 backbone with weights trained on ImageNet dataset and only train the dense layers and softmax layer that we have added.


In [ ]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(512,activation='relu')(x) #dense layer 1
x=tf.keras.layers.Dense(256,activation='relu')(x) #dense layer 2
preds=tf.keras.layers.Dense(num_classes,activation='softmax')(x) #final layer with softmax activation

model=tf.keras.Model(inputs=base_model.input,outputs=preds)

In [ ]:
#Freeze layers from MobileNetV2 backbone (not to be trained)
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
#Prepare the training dataset as a data generator object
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('images',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
# Build the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

## Train the Model


In [ ]:
start_datetime = datetime.now()

set_random_seed(2)
step_size_train=5
epochs=4
log_file = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=epochs)

end_datetime = datetime.now()
print('Training Duration: ' + str(end_datetime-start_datetime))

## Model Performance


In [ ]:
# Model accuracy and loss vs epoch
plt.plot(log_file.history['acc'], '-bo', label="train_accuracy")
plt.plot(log_file.history['loss'], '-r*', label="train_loss")
plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend(loc='center right')
plt.show()

## Let's Report Our Results Back to CV Studio


In [ ]:
parameters = {
    'epochs': epochs
    'step_size_train': step_size_train,
    'class_indices': train_generator.class_indices
}
accuracy = {
    'accuracy': log_file.history['acc'],
    'loss': log_file.history['loss']
}
result = cvstudioClient.report(started=start_datetime, completed=end_datetime, parameters=parameters, accuracy=accuracy)

if result.ok:
    print('Congratulations your results have been reported back to CV Studio!')

In [ ]:
# Save the model to a file
model.save('my_model.h5')

In [ ]:
parameters = {
    'epochs': epochs
    'step_size_train': step_size_train,
    
}
# Now let's save the model back to CV Studio
result = cvstudioClient.uploadModel('my_model.h5', parameters)


# Thanks for completing this notebook!
